# CAO Points analysis
***

## Purpose
##### The purpose of this notebook is to analyse the CAO points required for Level 8 courses in 3rd level institutions in Ireland. 
##### The data used in this analysis can be found [here](http://www.cao.ie/index.php?page=points&p=2021).

### Library imports
##### The libraries used in the analysis are numpy, pyplot and Element Tree

In [28]:
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import re
import requests as rq
import pandas as pd
import urllib.request as urlrq

## CAO Points 2021 analysis
***

In [29]:
# Get CAO points URL
resp = rq.get('http://www2.cao.ie/points/l8.php')

In [30]:
# Check request status
resp

<Response [200]>

In [31]:
def convert_points(s):
    portfolio = ''
    if s[0] == '#':
        portfolio = '#'
    random = ''
    if s[-1] == '*':
        random = '*'
    points = ''
    for i in s:
        if i.isdigit():
            points = points + i
    return[points, portfolio, random]

### Save Dataset
##### Using the datetime module, create a unique file name for the dataset and save it to back it up. 

In [32]:
import datetime as dt
now = dt.datetime.now()
nowstr = now.strftime('%Y%m%d_%H%M%S')
filenameHTML = 'data/cao2021_html_' + nowstr + '.html'

##### The encoding given by the server is not the correct encoding that the html is in. The following lines of code correct this. 

In [33]:
original_encoding = resp.encoding
resp.encoding = 'cp1252'
print(original_encoding)
print(resp.encoding)

iso-8859-1
cp1252


##### Now that the encoding has been corrected, the data can be saved to a csv file.

In [34]:
with open(filenameHTML, 'w') as f:
    f.write(resp.text)

In [35]:
# Compile regular expression for matching lines
re_course = re.compile(r'([A-Z]{2}[0-9]{3})(.*)')

In [39]:
# iterate through lines in the response
filenameCSV = 'data/cao2021_csv_' + nowstr + '.csv'

num_lines = 0 

with open(filenameCSV, 'w') as f:
    # Write the headers
    f.write(','.join(['course_code', 'title', 'pointsR1', 'pointsR2']) + '\n')
    for line in resp.iter_lines():
        dline = line.decode('cp1252')
        # Get only the lines with courses and points
        if re_course.fullmatch(dline):
            num_lines += 1
            course_code = dline[:5]
            course_title = dline[7:57]
            course_points = re.split(' +', dline[60:])
            if len(course_points) != 2:
                course_points = course_points[:2]
            linesplit = [course_code, course_title, course_points[0], course_points[1]]
            # linesplit = re.split('  +', dline)
            f.write(','.join(linesplit) + '\n')
            #print(line)
#print(f"Total number of lines is {num_lines}")

df2021 = pd.read_csv(filenameCSV, encoding='cp1252')
df2021

,course_code,title,pointsR1,pointsR2
0,AL801,Software Design for Virtual Reality and Gaming...,300,NaN
1,AL802,Software Design in Artificial Intelligence for...,313,NaN
2,AL803,Software Design for Mobile Apps and Connected ...,350,NaN
3,AL805,Computer Engineering for Network Infrastructur...,321,NaN
4,AL810,Quantity Surveying ...,328,NaN
...,...,...,...,...
944,WD211,Creative Computing ...,270,NaN
945,WD212,Recreation and Sport Management ...,262,NaN
946,WD230,Mechanical and Manufacturing Engineering ...,230,230
947,WD231,Early Childhood Care and Education ...,266,NaN


## CAO Points 2020 data import
***

In [11]:
# Create file path for raw data
path2020 = 'data/cao2020_' + nowstr + '.xlsx'

In [12]:
# Save raw data locally
urlrq.urlretrieve('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx', path2020)

('data/cao2020_20220102_143007.xlsx',
 <http.client.HTTPMessage at 0x1bcd5a8c340>)

In [13]:
df2020 = pd.read_excel('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx', skiprows=10)

In [14]:
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df2020.iloc[754]

CATEGORY (i.e.ISCED description)    Engineering and engineering trades
COURSE TITLE                                    Mechanical Engineering
COURSE CODE2                                                     LC288
R1 POINTS                                                          347
R1 Random *                                                        NaN
R2 POINTS                                                          346
R2 Random*                                                         NaN
EOS                                                                346
EOS Random *                                                       NaN
EOS Mid-point                                                      415
LEVEL                                                                8
HEI                                   Limerick Institute of Technology
Test/Interview #                                                   NaN
avp                                                                NaN
v     

In [16]:
# Create file path for pandas dataframe data
path = 'data/cao2020_' + nowstr + '.csv'

In [17]:
# Save data in pandas dataframe to a csv file locally
df2020.to_csv(path)

## CAO Points 2019 data import
***

##### The data for 2019 is stored in a PDF file. To allow analysis on this data, the following steps were taken:
1. Download PDF
2. Open PDF using MS Word
3. Save document as .docx file
4. Save a copy of the .docx file for editing
5. Remove the headers and footers
6. Remove pre-amble text
7. Select all of the text in the document
8. Copy and paste to Notepad
9. Remove HEI headings and add HEI name to each course line.
10. Remove blank lines
11. Replace backticks with apostrophes
12. Replace double tabs with single tabs
13. Removed any additional tabs at the end of lines

In [18]:
df2019 = pd.read_csv('data/cao2019_20220102_0721_edited.csv', sep='\t')

In [19]:
df2019

,HEI,Course Code,INSTITUTION and COURSE,EOS,Mid
0,Athlone Institute of Technology,AL801,Software Design with Virtual Reality and Gaming,304,328.0
1,Athlone Institute of Technology,AL802,Software Design with Cloud Computing,301,306.0
2,Athlone Institute of Technology,AL803,Software Design with Mobile Apps and Connected...,309,337.0
3,Athlone Institute of Technology,AL805,Network Management and Cloud Infrastructure,329,442.0
4,Athlone Institute of Technology,AL810,Quantity Surveying,307,349.0
...,...,...,...,...,...
925,Waterford Institute of Technology,WD200,Arts (options),221,296.0
926,Waterford Institute of Technology,WD210,Software Systems Development,271,329.0
927,Waterford Institute of Technology,WD211,Creative Computing,275,322.0
928,Waterford Institute of Technology,WD212,Recreation and Sport Management,274,311.0


## Concatenating the Data
***

In [20]:
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df2019

,HEI,Course Code,INSTITUTION and COURSE,EOS,Mid
0,Athlone Institute of Technology,AL801,Software Design with Virtual Reality and Gaming,304,328.0
1,Athlone Institute of Technology,AL802,Software Design with Cloud Computing,301,306.0
2,Athlone Institute of Technology,AL803,Software Design with Mobile Apps and Connected...,309,337.0
3,Athlone Institute of Technology,AL805,Network Management and Cloud Infrastructure,329,442.0
4,Athlone Institute of Technology,AL810,Quantity Surveying,307,349.0
...,...,...,...,...,...
925,Waterford Institute of Technology,WD200,Arts (options),221,296.0
926,Waterford Institute of Technology,WD210,Software Systems Development,271,329.0
927,Waterford Institute of Technology,WD211,Creative Computing,275,322.0
928,Waterford Institute of Technology,WD212,Recreation and Sport Management,274,311.0


In [53]:
courses2021 = df2021[['course_code', 'title']]
courses2021

,course_code,title
0,AL801,Software Design for Virtual Reality and Gaming...
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructur...
4,AL810,Quantity Surveying ...
...,...,...
944,WD211,Creative Computing ...
945,WD212,Recreation and Sport Management ...
946,WD230,Mechanical and Manufacturing Engineering ...
947,WD231,Early Childhood Care and Education ...


In [54]:
courses2020 = df2020[['COURSE CODE2', 'COURSE TITLE']]
courses2020.columns = ['course_code', 'title']
courses2021

,course_code,title
0,AL801,Software Design for Virtual Reality and Gaming...
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructur...
4,AL810,Quantity Surveying ...
...,...,...
944,WD211,Creative Computing ...
945,WD212,Recreation and Sport Management ...
946,WD230,Mechanical and Manufacturing Engineering ...
947,WD231,Early Childhood Care and Education ...


In [61]:
# Combines courses2021 and courses2020 into one dataframe, allcourses
allcourses = pd.concat([courses2021, courses2020])

# Returns a dataframe of all extra copies of row with the same course_code
allcourses[allcourses.duplicated(subset='course_code')]


,course_code,title
0,AC120,International Business
1,AC137,Liberal Arts
2,AD101,"First Year Art & Design (Common Entry,portfolio)"
3,AD102,Graphic Design and Moving Image Design (portfo...
4,AD103,Textile & Surface Design and Jewellery & Objec...
...,...,...
1455,WD200,Arts (options)
1460,WD210,Software Systems Development
1461,WD211,Creative Computing
1462,WD212,Recreation and Sport Management


In [64]:
# Returns allcourses dataframe with duplicate course code removed.
allcourses.drop_duplicates(subset='course_code', inplace=True)

In [65]:
allcourses

,course_code,title
0,AL801,Software Design for Virtual Reality and Gaming...
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructur...
4,AL810,Quantity Surveying ...
...,...,...
1449,WD188,Applied Health Care
1456,WD205,Molecular Biology with Biopharmaceutical Science
1457,WD206,Electronic Engineering
1458,WD207,Mechanical Engineering
